**Support code for Exploratory Report** <br>
https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca

In [1]:
#!pip install web3

In [2]:
import json
from datetime import datetime
from decimal import Decimal
import pandas as pd
from web3 import Web3
from web3.exceptions import (InvalidEventABI, LogTopicError, MismatchedABI)
from web3._utils.events import get_event_data
from eth_utils import (encode_hex, event_abi_to_log_topic)


# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
print ('Ethereum connected:', w3_eth.isConnected())


Ethereum connected: True


In [3]:
abi = json.loads("""[
                     {"name":"decimals",
                      "inputs":[],"outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view",
                      "type":"function"},
                     {"name":"AnswerUpdated",
                      "anonymous":false,
                      "inputs":[{"indexed":true,"internalType":"int256","name":"current","type":"int256"},{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"updatedAt","type":"uint256"}],
                      "type":"event"}   
                    ]
                """)

pricefeed_contracts = {
     'eth-usd' : [w3_eth, Web3.toChecksumAddress('0x37bC7498f4FF12C19678ee8fE19d713b87F6a9e6'), 0.005, 'usd', 3600]
    
    ,'btc-eth' : [w3_eth, Web3.toChecksumAddress('0x81076d6ff2620ea9dd7ba9c1015f0d09a3a732e6'), 0.02, 'eth', 86400]
    ,'btc-usd' : [w3_eth, Web3.toChecksumAddress('0xAe74faA92cB67A95ebCAB07358bC222e33A34dA7'), 0.005, 'usd', 3600]
    
    ,'usdc-eth': [w3_eth, Web3.toChecksumAddress('0xe5bbbdb2bb953371841318e1edfbf727447cef2e'), 0.01, 'eth', 86400]
    ,'usdc-usd': [w3_eth, Web3.toChecksumAddress('0x789190466E21a8b78b8027866CBBDc151542A26C'), 0.0025, 'usd', 86400]
    
    ,'dai-eth' : [w3_eth, Web3.toChecksumAddress('0x158228e08c52f3e2211ccbc8ec275fa93f6033fc'), 0.01, 'eth', 86400]
    ,'dai-usd' : [w3_eth, Web3.toChecksumAddress('0xDEc0a100eaD1fAa37407f0Edc76033426CF90b82'), 0.0025, 'usd', 3600]
    
    ,'1inch-eth': [w3_eth, Web3.toChecksumAddress('0xb2f68c82479928669b0487d1daed6ef47b63411e'), 0.02, 'eth', 86400]
    ,'1inch-usd': [w3_eth, Web3.toChecksumAddress('0xd2bdD1E01fd2F8d7d42b209c111c7b32158b5a42'), 0.01, 'usd', 86400]
    
    ,'aave-eth': [w3_eth, Web3.toChecksumAddress('0xdf0da6b3d19e4427852f2112d0a963d8a158e9c7'), 0.02, 'eth', 86400]
    ,'aave-usd': [w3_eth, Web3.toChecksumAddress('0xe3f0dEdE4B499c07e12475087AB1A084b5F93bc0'), 0.01, 'usd', 3600]
    
    ,'comp-eth': [w3_eth, Web3.toChecksumAddress('0x9d6acd34d481512586844fd65328bd358d306752'), 0.02, 'eth', 86400]     
    ,'comp-usd': [w3_eth, Web3.toChecksumAddress('0x6eaC850f531d0588c0114f1E93F843B78669E6d2'), 0.01, 'usd', 3600]     
    
    ,'dpi-eth': [w3_eth, Web3.toChecksumAddress('0x36e4f71440edf512eb410231e75b9281d4fcfc4c'), 0.02, 'eth', 86400]     
    ,'dpi-usd': [w3_eth, Web3.toChecksumAddress('0x68f1b8317c19ff02fb68a8476c1d3f9fc5139c0a'), 0.01, 'usd', 3600]     

    ,'fei-eth': [w3_eth, Web3.toChecksumAddress('0x4be991b4d560bba8308110ed1e0d7f8da60acf6a'), 0.02, 'eth', 86400] 
    ,'fei-usd': [w3_eth, Web3.toChecksumAddress('0x1D244648d5a63618751d006886268Ae3550d0Dfd'), 0.01, 'usd', 3600] 
    
    ,'link-eth': [w3_eth, Web3.toChecksumAddress('0xbba12740de905707251525477bad74985dec46d2'), 0.01, 'eth', 21600] 
    ,'link-usd': [w3_eth, Web3.toChecksumAddress('0xDfd03BfC3465107Ce570a0397b247F546a42D0fA'), 0.01, 'usd', 3600] 

    ,'snx-eth': [w3_eth, Web3.toChecksumAddress('0xbafe3cb0e563e914806a99d547bdbf2cfcf5fdf6'), 0.02, 'eth', 86400] 
    ,'snx-usd': [w3_eth, Web3.toChecksumAddress('0x06ce8Be8729B6bA18dD3416E3C223a5d4DB5e755'), 0.01, 'usd', 86400] 

    ,'uni-eth': [w3_eth, Web3.toChecksumAddress('0xc1d1d0da0fcf78157ea25d0e64e3be679813a1f7'), 0.02, 'eth', 86400] 
    ,'uni-usd': [w3_eth, Web3.toChecksumAddress('0x68577f915131087199Fe48913d8b416b3984fd38'), 0.01, 'usd', 3600] 

    ,'yfi-eth': [w3_eth, Web3.toChecksumAddress('0xaa5aa80e416f9d32ffe6c390e24410d02d203f70'), 0.01, 'eth', 86400]
    ,'yfi-usd': [w3_eth, Web3.toChecksumAddress('0x8a4D74003870064d41D4f84940550911FBfCcF04'), 0.01, 'usd', 3600]
    
    ,'luna-eth': [w3_eth, Web3.toChecksumAddress('0x1a8ac67a1b64f7fd71bb91c21581f036abe6aec2'), 0.02, 'eth', 86400]
    
    ,'ftm-eth': [w3_eth, Web3.toChecksumAddress('0xbdb80d19dea36eb7f63bdfd2bdd4033b2b7e8e4d'), 0.03, 'eth', 86400]
     
    ,'sushi-eth': [w3_eth, Web3.toChecksumAddress('0xd01bbb3afed2cb5ca92ca3834d441dc737f0da70'), 0.02, 'eth', 86400]
    ,'sushi-usd': [w3_eth, Web3.toChecksumAddress('0x7213536a36094cD8a768a5E45203Ec286Cba2d74'), 0.01, 'usd', 3600]
    
    ,'crv-eth': [w3_eth, Web3.toChecksumAddress('0x7f67ca2ce5299a67acd83d52a064c5b8e41ddb80'), 0.02, 'eth', 86400]
    ,'crv-usd': [w3_eth, Web3.toChecksumAddress('0xb4c4a493AB6356497713A78FFA6c60FB53517c63'), 0.01, 'usd', 86400]
    
    ,'cvx-usd': [w3_eth, Web3.toChecksumAddress('0x8d73ac44bf11cadcdc050bb2bccae8c519555f1a'), 0.02, 'usd', 86400] 
    
    ,'ldo-eth': [w3_eth, Web3.toChecksumAddress('0x7898AcCC83587C3C55116c5230C17a6Cd9C71bad'), 0.02, 'eth', 86400] 
    
    ,'steth-eth': [w3_eth, Web3.toChecksumAddress('0x716BB759A5f6faCdfF91F0AfB613133d510e1573'), 0.02, 'eth', 86400] 
    ,'steth-usd': [w3_eth, Web3.toChecksumAddress('0xdA31bc2B08F22AE24aeD5F6EB1E71E96867BA196'), 0.01, 'usd', 3600] 
    
    ,'frax-eth': [w3_eth, Web3.toChecksumAddress('0x56f98706C14DF5C290b02Cec491bB4c20834Bb51'), 0.02, 'eth', 86400] 
    ,'frax-usd': [w3_eth, Web3.toChecksumAddress('0x61eB091ea16A32ea5B880d0b3D09d518c340D750'), 0.01, 'usd', 3600] 

    ,'lusd-usd': [w3_eth, Web3.toChecksumAddress('0x27b97a63091d185cE056e1747624b9B92BAAD056'), 0.01, 'usd', 3600] 

    ,'susd-eth': [w3_eth, Web3.toChecksumAddress('0x45bb69B89D60878d1e42522342fFCa9F2077dD84'), 0.01, 'eth', 86400] 
    ,'susd-usd': [w3_eth, Web3.toChecksumAddress('0x1187272A0E3A603eC4734CeC73a0880055eCC593'), 0.005, 'usd', 86400] 

    ,'gusd-eth': [w3_eth, Web3.toChecksumAddress('0x9c2C487DAd6C8e5bb49dC6908a29D95a234FaAd8'), 0.02, 'eth', 86400] 
    ,'gusd-usd': [w3_eth, Web3.toChecksumAddress('0x6a805f2580b8D75d40331c26C074c2c42961E7F2'), 0.005, 'usd', 86400] 

    ,'usdt-eth': [w3_eth, Web3.toChecksumAddress('0x7De0d6fce0C128395C488cb4Df667cdbfb35d7DE'), 0.01, 'eth', 86400] 
    ,'usdt-usd': [w3_eth, Web3.toChecksumAddress('0xa964273552C1dBa201f5f000215F5BD5576e8f93'), 0.0025, 'usd', 86400] 
    }

config = {'Ethereum':pricefeed_contracts, 
         }

In [4]:
def get_event_abi(abi, abi_name, abi_type = 'event'):
    l = [x for x in abi if x['type'] == abi_type and x['name']==abi_name]
    return l[0]
def get_logs (w3, contract_address, topics, events, from_block=0, to_block=None):

    if not to_block:
        to_block = w3.eth.get_block('latest').number
    
    try:
        logs = w3.eth.get_logs({"address": contract_address
                               ,"topics":topics 
                               ,"fromBlock": from_block 
                               ,"toBlock": to_block
                               })
    except ValueError:
        logs = None
        from_block = 12*10**6
        batch_size = 10**6
        while from_block < to_block:
            print(datetime.utcnow(), f'from_block {from_block} to_block {from_block + batch_size}')
            batch_logs = w3.eth.get_logs({"address": contract_address
                                   ,"topics":topics 
                                   ,"fromBlock": from_block 
                                   ,"toBlock": from_block + batch_size # narrow down the range to avoid too many results error 
                                   })   
            if not logs:
                logs = batch_logs
            else:
                logs = logs + batch_logs
            if len(logs) > 0:    
                print(datetime.utcnow(), f'Total logs count {len(logs)}')  
            
            from_block = from_block + batch_size + 1
            
    all_events = []
    for l in logs:
        try:
            evt_topic0 = l['topics'][0].hex()            
            evt_abi = [x for x in events if encode_hex(event_abi_to_log_topic(x)) == evt_topic0][0]
            evt = get_event_data(w3.codec, evt_abi, l)
        except MismatchedABI: #if for some reason there are other events 
            pass
        all_events.append(evt)
    df = pd.DataFrame(all_events)
    return df

In [5]:
#from google.oauth2 import service_account
#import pandas_gbq

#try:
#    df_blocknum = pandas_gbq.read_gbq('select max(blockNumber) from gearbox.oracle_price_history', 
#                             project_id=gcp_project_id,
#                             progress_bar_type = None,)
#    from_block = int(df_blocknum.iloc[0,0]) + 1
#except pandas_gbq.exceptions.GenericGBQException:
#    print('The table does not exist?')
#    from_block = 0
from_block = 0
print(datetime.utcnow(),'from_block=', from_block)

df = pd.DataFrame()

event_AnswerUpdated = get_event_abi(abi, 'AnswerUpdated')
topic_AnswerUpdated  = encode_hex(event_abi_to_log_topic(event_AnswerUpdated))

for chain in config.keys():
    chain_contracts = config[chain]
    for ticker in chain_contracts.keys():
        print(f'{datetime.utcnow()} : {ticker.upper()}')
        w3 = chain_contracts[ticker][0]
        address = chain_contracts[ticker][1]
        threshold = chain_contracts[ticker][2]
        base = chain_contracts[ticker][3]
        heartbeat = chain_contracts[ticker][4]
        
        decimals = w3.eth.contract(address=address, abi=abi).functions.decimals().call()

        df_ticker = get_logs(w3, address, [topic_AnswerUpdated], [event_AnswerUpdated], from_block)
        df_ticker[['chain', 'ticker', 'threshold', 'base', 'heartbeat', 'decimals', 'type']] = [chain , ticker, threshold, base, heartbeat, decimals, 'direct']
        
        df_ticker['tx_hash']       = df_ticker['transactionHash'].apply(lambda x: x.hex())        
        df_ticker['updated_at']    = df_ticker['args'].apply(lambda x: pd.to_datetime(x['updatedAt'], unit='s'))
        df_ticker['price']         = df_ticker['args'].apply(lambda x: x['current'])
        df_ticker['price_decimal'] = df_ticker['price']/10**(decimals)
        df_ticker['deviation']     = df_ticker['price']/df_ticker['price'].shift(1) - 1
        
        df = df.append(df_ticker[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'decimals', 'blockNumber','tx_hash']], ignore_index=True)
        print(f'''{datetime.utcnow()} : {ticker.upper()}: observation count = {len(df_ticker)}, price range: {df_ticker['price_decimal'].min()}-{df_ticker['price_decimal'].max()}''')
        
        if ticker == 'eth-usd':
            df_eth_usd = df_ticker
        elif base == 'usd': # usd2eth cross rate
            ticker = ticker.replace('-usd','-eth')
            if ticker not in chain_contracts.keys():
                print(f'{datetime.utcnow()}: cross rate {ticker.upper()}')
                df_cross = pd.DataFrame()
                for k, t in df_ticker.iterrows(): 
                    e = df_eth_usd[df_eth_usd['updated_at'] <= t['updated_at']]
                    if len(e) > 0:
                        i = (t['updated_at'] - e['updated_at']).idxmin() 
                        e_price_decimal, e_decimals = e.loc[i][['price_decimal','decimals']]
                        t.ticker = ticker
                        t.price_decimal = t.price_decimal/e_price_decimal
                        t.decimals = 18
                        t.type = 'cross'
                        t.price = int(t.price_decimal*10**(t.decimals))
                        df_cross = df_cross.append(t)
                df_cross['deviation']     = df_cross['price']/df_cross['price'].shift(1) - 1
                df = df.append(df_cross[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'decimals', 'blockNumber','tx_hash']], ignore_index=True)
                print(f'''{datetime.utcnow()}: cross rate {ticker.upper()}: observation count = {len(df_cross)}, price range: {df_cross['price_decimal'].min()}-{df_cross['price_decimal'].max()}''')
        elif base == 'eth': #eth2usd cross rate
            ticker = ticker.replace('-eth','-usd')
            if ticker not in chain_contracts.keys():
                print(f'{datetime.utcnow()}: cross rate {ticker.upper()}')
                df_cross = pd.DataFrame()
                for k, t in df_ticker.iterrows(): 
                    e = df_eth_usd[df_eth_usd['updated_at'] <= t['updated_at']]
                    if len(e) > 0:
                        i = (t['updated_at'] - e['updated_at']).idxmin() 
                        e_price_decimal, e_decimals = e.loc[i][['price_decimal','decimals']]
                        t.ticker = ticker
                        t.decimals = e_decimals
                        t.type = 'cross'
                        t.price_decimal = t.price_decimal*e_price_decimal
                        t.price = int(t.price_decimal*10**(t.decimals))
                        df_cross = df_cross.append(t)
                df_cross['deviation']     = df_cross['price']/df_cross['price'].shift(1) - 1
                df = df.append(df_cross[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'blockNumber','tx_hash']], ignore_index=True)
                print(f'''{datetime.utcnow()}: cross rate {ticker.upper()}: observation count = {len(df_cross)}, price range: {df_cross['price_decimal'].min()}-{df_cross['price_decimal'].max()}''')
                
df = df.sort_values(by = 'updated_at').reset_index(drop=True)
pd.set_option("precision", 18)

2022-04-25 07:01:47.005845 from_block= 0
2022-04-25 07:01:47.007539 : ETH-USD
2022-04-25 07:01:48.520662 from_block 12000000 to_block 13000000
2022-04-25 07:01:54.024503 Total logs count 8469
2022-04-25 07:01:54.024654 from_block 13000001 to_block 14000001
2022-04-25 07:01:58.477644 Total logs count 17305
2022-04-25 07:01:58.477914 from_block 14000002 to_block 15000002
2022-04-25 07:02:02.222522 Total logs count 22855
2022-04-25 07:02:12.455667 : ETH-USD: observation count = 22855, price range: 1720.62935399-4850.92
2022-04-25 07:02:12.456135 : BTC-ETH
2022-04-25 07:02:14.287986 : BTC-ETH: observation count = 779, price range: 11.45671593-33.65841927889143
2022-04-25 07:02:14.288553 : BTC-USD
2022-04-25 07:02:18.496587 from_block 12000000 to_block 13000000
2022-04-25 07:02:21.350502 Total logs count 6451
2022-04-25 07:02:21.351012 from_block 13000001 to_block 14000001
2022-04-25 07:02:25.373923 Total logs count 13807
2022-04-25 07:02:25.374042 from_block 14000002 to_block 15000002
2022

2022-04-25 07:07:08.911753 : CVX-USD: observation count = 798, price range: 15.32810389-47.16700316
2022-04-25 07:07:08.912207: cross rate CVX-ETH
2022-04-25 07:07:17.898803: cross rate CVX-ETH: observation count = 798, price range: 0.005740313831078381-0.014475643754177692
2022-04-25 07:07:17.899230 : LDO-ETH
2022-04-25 07:07:24.472320 : LDO-ETH: observation count = 4250, price range: 0.000473154043167557-0.00225298
2022-04-25 07:07:24.472737: cross rate LDO-USD
2022-04-25 07:08:04.885663: cross rate LDO-USD: observation count = 4127, price range: 1.196048253399814-7.238858687204216
2022-04-25 07:08:04.886081 : STETH-ETH
2022-04-25 07:08:05.834413 : STETH-ETH: observation count = 246, price range: 0.98081098-1.0000460908852682
2022-04-25 07:08:05.834780 : STETH-USD
2022-04-25 07:08:15.286691 : STETH-USD: observation count = 6725, price range: 2187.0133105-4813.89441143
2022-04-25 07:08:15.287199 : FRAX-ETH
2022-04-25 07:08:17.454834 : FRAX-ETH: observation count = 1346, price range: 0

In [10]:
display(df.head())
display(df.tail())

,ticker,updated_at,chain,price,price_decimal,type,deviation,threshold,base,heartbeat,decimals,blockNumber,tx_hash
0,comp-eth,2021-02-18 09:09:54,Ethereum,244378668045116032.0,0.244378668045116043,direct,NaN,0.020000000000000000,eth,86400.0,18.0,11879906.0,0x9610367f2cd00cba48361195cb211f54e5775cf54efc...
1,crv-eth,2021-02-18 10:43:05,Ethereum,1486000000000000.0,0.001486000000000000,direct,NaN,0.020000000000000000,eth,86400.0,18.0,11880352.0,0xa102d125a730c7186a458749e3a8de6323713567c6fa...
2,snx-eth,2021-02-18 10:55:03,Ethereum,12479365000000000.0,0.012479364999999999,direct,NaN,0.020000000000000000,eth,86400.0,18.0,11880406.0,0x7537d4fce2d92483de4fd133947ec2cde6883f839685...
3,crv-eth,2021-02-18 11:47:08,Ethereum,1454500000000000.0,0.001454500000000000,direct,-0.021197846567967749,0.020000000000000000,eth,86400.0,18.0,11880656.0,0xb170655b26a54ea69deb98102105e6b11a686338dcb1...
4,ftm-eth,2021-02-18 12:23:53,Ethereum,108390000000000.0,0.000108390000000000,direct,NaN,0.029999999999999999,eth,86400.0,18.0,11880835.0,0x3a973394035ccfd333f19fbeb09003373e22a269331e...


,ticker,updated_at,chain,price,price_decimal,type,deviation,threshold,base,heartbeat,decimals,blockNumber,tx_hash
245931,lusd-usd,2022-04-25 07:04:23,Ethereum,99931314.0,0.999313140000000044,direct,-0.001297706270936394,0.01,usd,3600.0,8.0,14652356.0,0x2dbd5cee16a336a2b54bef77dbf323c2246d69f88f12...
245932,lusd-eth,2022-04-25 07:04:23,Ethereum,353371829104087.0,0.000353371829104088,cross,0.007896129519532114,0.01,usd,3600.0,18.0,14652356.0,0x2dbd5cee16a336a2b54bef77dbf323c2246d69f88f12...
245933,crv-usd,2022-04-25 07:05:25,Ethereum,254174851.0,2.541748510000000127,direct,-0.011286353287207840,0.01,usd,86400.0,8.0,14652360.0,0x9309fbfd5aec1d76cd43cd11abaedf6c1c92377ca5c8...
245934,frax-eth,2022-04-25 07:06:34,Ethereum,353880000000000.0,0.000353880000000000,direct,0.021202829067735740,0.02,eth,86400.0,18.0,14652364.0,0x29dd29e2006d96a14c6ffe746b527c6cf608d83b5aa1...
245935,usdt-eth,2022-04-25 07:07:27,Ethereum,355215739093379.0,0.000355215739093379,direct,0.012501037805715098,0.01,eth,86400.0,18.0,14652368.0,0xd9a30b6f5e801d4a196121637babee3c4fe2efab1e45...


**Write data frame to GCP BigQuery**

In [7]:
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'
if len(df)>0:
    numeric_cols = [x for x in df.columns if x in ['price', 'price_decimal' ,'deviation']]
    df[numeric_cols] = df[numeric_cols].astype('float64')

    pandas_gbq.to_gbq(df, 
                      'gearbox.oracle_price_history',
                      project_id=gcp_project_id,
                      if_exists = ('replace' if from_block==0 else 'append'), 
                      progress_bar = False)
    print(datetime.utcnow(), 'gearbox.oracle_price_history, insert done')
else:
    print('No new events since block', from_block)

2022-04-25 07:10:06.996062 gearbox.oracle_price_history, insert done


Exploratory report:

https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca